In [1]:
%%bash
pip install stemming

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stemming: filename=stemming-1.0.1-py3-none-any.whl size=11138 sha256=c62de03d2165653f67c487b49be6adaf09e7562fcc921c8d117e4b0d596c3ab2
  Stored in directory: /root/.cache/pip/wheels/6b/e5/e2/c52ebc0a5b53fd82b00cc385e57bb1c90bd50e5f54ddbc06d1
Successfully built stemming


In [2]:
import string
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from stemming.porter2 import stem
import networkx 
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.pylab as plt
from IPython.display import Image
from community import community_louvain
from collections import Counter
from itertools import chain
#pyo.init_notebook_mode()
import plotly.io as pio
pio.renderers.default = 'browser'
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
# Open the file to read Amazon Product Metadata
df = open ('/content/drive/MyDrive/GNN/amazon-meta.txt', 'r', encoding='utf-8', errors= 'ignore')

In [37]:
# Initialize a nested product dictionary that will hold cleaned up amazon product data. 
amazonProducts= {}

In [38]:
# Read the data from the Amazon file and fill the amazonProducts nested dictionary
(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)

for line in df:
    line = line.strip()
    if(line.startswith("Id")): # a product block started
        Id = line[3:].strip()
    elif(line.startswith("ASIN")):
        ASIN = line[5:].strip()
    elif(line.startswith("title")):
        Title = line[6:].strip()
        Title = ' '.join(Title.split())
        continue
    elif(line.startswith("group")):
        Group = line[6:].strip()
    elif(line.startswith("salesrank")):
        SalesRank = line[10:].strip()
    elif(line.startswith("similar")):
        ls = line.split()
        no_sim = ls[1:2]
        Copurchased = ' '.join([c for c in ls[2:]])
    elif(line.startswith("categories")):
        ls = line.split()
        Categories = ' '.join((df.readline()).lower() for i in range(int(ls[1].strip())))
        Categories = re.compile('[%s]' % re.escape(string.digits+string.punctuation)).sub(' ',Categories)
        Categories = ' '.join(set(Categories.split())-set(stopwords.words("english")))
        Categories = ' '.join(stem(word) for word in Categories.split())
    elif(line.startswith("reviews")):
        ls = line.split()
        TotalReviews = ls[2].strip()
        AvgRating = ls[7].strip() # a product block ended
    elif (line==""): # write out fields to amazonProducts dictionary
        try:
            MetaData = {}
            if (ASIN != ""):
                amazonProducts[ASIN] = MetaData
            MetaData['Id'] = Id
            MetaData['Title'] = Title
            MetaData['Categories'] = ' '.join(set(Categories.split()))
            MetaData['Group'] = Group
            MetaData['Copurchased'] = Copurchased
            MetaData['SalesRank'] = int(SalesRank)
            MetaData['TotalReviews'] = int(TotalReviews)
            MetaData['AvgRating'] = float(AvgRating)
            #MetaData['DegreeCentrality'] = DegreeCentrality
            #MetaData['ClusteringCoeff'] = ClusteringCoeff
            MetaData['NoSim'] = no_sim
        except NameError:
            continue
        (Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)
df.close

<function TextIOWrapper.close>

In [39]:
for asin, metadata in amazonProducts.items():
    amazonProducts[asin]['Copurchased']= ' '.join([cp for cp in metadata['Copurchased'].split() if cp in amazonProducts.keys()])

In [40]:
df = pd.DataFrame.from_dict(amazonProducts)

In [41]:
df = df.T

In [42]:
df

,Id,Title,Categories,Group,Copurchased,SalesRank,TotalReviews,AvgRating,NoSim
0771044445,0,,,,,0,0,0.0,[5]
0827229534,1,Patterns of Preaching: A Sermon Sampler,christian spiritu religion clergi preach book ...,Book,0804215715 156101074X 0687023955 0687074231 08...,396585,2,5.0,[5]
0738700797,2,Candlemas: Feast of Flames,spiritu religion earth witchcraft book wicca b...,Book,0738700827 1567184960 1567182836 0738700525 07...,168596,12,4.5,[5]
0486287785,3,World War II Allied Fighter Planes Trading Cards,hobbi garden craft general home book subject,Book,,1270652,1,5.0,[0]
0842328327,4,Life Application Bible Commentary: 1 and 2 Tim...,spiritu sacr applic book refer religion guid d...,Book,0842328130 0842330313 0842328610 0842328572,631289,1,4.0,[5]
...,...,...,...,...,...,...,...,...,...
B000059TOC,548547,The Drifter,horror actor categori ann director dvd featur ...,DVD,630366704X,0,1,5.0,[5]
B00006JBIX,548548,The House Of Morecock,anim distribut comedi intern special dvd genr ...,DVD,B00004WZQN,0,8,3.0,[5]
0879736836,548549,Catholic Bioethics and the Gift of Human Life,histori christian spiritu religion general soc...,Book,1580510469 0896229939,0,1,4.0,[5]
B00008DDST,548550,"1, 2, 3 Soleils: Taha, Khaled, Faudel",video deal categori dvd special genr featur ge...,DVD,,0,3,5.0,[5]


In [44]:
def getclean(l):
    p = int(l[0]) 
    return p  
    

In [45]:
df

,Id,Title,Categories,Group,Copurchased,SalesRank,TotalReviews,AvgRating,NoSim
0771044445,0,,,,,0,0,0.0,[5]
0827229534,1,Patterns of Preaching: A Sermon Sampler,christian spiritu religion clergi preach book ...,Book,0804215715 156101074X 0687023955 0687074231 08...,396585,2,5.0,[5]
0738700797,2,Candlemas: Feast of Flames,spiritu religion earth witchcraft book wicca b...,Book,0738700827 1567184960 1567182836 0738700525 07...,168596,12,4.5,[5]
0486287785,3,World War II Allied Fighter Planes Trading Cards,hobbi garden craft general home book subject,Book,,1270652,1,5.0,[0]
0842328327,4,Life Application Bible Commentary: 1 and 2 Tim...,spiritu sacr applic book refer religion guid d...,Book,0842328130 0842330313 0842328610 0842328572,631289,1,4.0,[5]
...,...,...,...,...,...,...,...,...,...
B000059TOC,548547,The Drifter,horror actor categori ann director dvd featur ...,DVD,630366704X,0,1,5.0,[5]
B00006JBIX,548548,The House Of Morecock,anim distribut comedi intern special dvd genr ...,DVD,B00004WZQN,0,8,3.0,[5]
0879736836,548549,Catholic Bioethics and the Gift of Human Life,histori christian spiritu religion general soc...,Book,1580510469 0896229939,0,1,4.0,[5]
B00008DDST,548550,"1, 2, 3 Soleils: Taha, Khaled, Faudel",video deal categori dvd special genr featur ge...,DVD,,0,3,5.0,[5]


In [46]:
df['NoSim'] = df['NoSim'].apply(lambda x: getclean(x))

In [54]:
df.tail()

,Id,Title,Categories,Group,Copurchased,SalesRank,TotalReviews,AvgRating,NoSim,ASIN
B000059TOC,548547,The Drifter,horror actor categori ann director dvd featur ...,DVD,630366704X,0,1,5.0,5,B000059TOC
B00006JBIX,548548,The House Of Morecock,anim distribut comedi intern special dvd genr ...,DVD,B00004WZQN,0,8,3.0,5,B00006JBIX
0879736836,548549,Catholic Bioethics and the Gift of Human Life,histori christian spiritu religion general soc...,Book,1580510469 0896229939,0,1,4.0,5,0879736836
B00008DDST,548550,"1, 2, 3 Soleils: Taha, Khaled, Faudel",video deal categori dvd special genr featur ge...,DVD,,0,3,5.0,5,B00008DDST
B00005MHUG,548551,That Travelin' Two-Beat/Sings the Great Countr...,tradit nostalgia classic broadway pop general ...,Music,B00005O6KL B0000634HG,0,1,5.0,5,B00005MHUG


In [56]:
df1.head()

,Title,Categories,Group,Copurchased,SalesRank,TotalReviews,AvgRating,NoSim,ASIN
Id,,,,,,,,,
0,,,,,0,0,0.0,5,0771044445
1,Patterns of Preaching: A Sermon Sampler,christian spiritu religion clergi preach book ...,Book,0804215715 156101074X 0687023955 0687074231 08...,396585,2,5.0,5,0827229534
2,Candlemas: Feast of Flames,spiritu religion earth witchcraft book wicca b...,Book,0738700827 1567184960 1567182836 0738700525 07...,168596,12,4.5,5,0738700797
3,World War II Allied Fighter Planes Trading Cards,hobbi garden craft general home book subject,Book,,1270652,1,5.0,0,0486287785
4,Life Application Bible Commentary: 1 and 2 Tim...,spiritu sacr applic book refer religion guid d...,Book,0842328130 0842330313 0842328610 0842328572,631289,1,4.0,5,0842328327


In [53]:
df['ASIN'] = df.index

In [55]:
df1 = df.set_index('Id')

In [64]:
df1.to_csv('/content/drive/MyDrive/BAJAJ/GNN_datasets/pdssimilar2.csv')

In [ ]:
df.to_csv('/content/drive/MyDrive/similar.csv')

In [58]:
#create a product copurchase graph for analysis
#the graph nodes are product ASINs, the graph edge exists if two products were copurchased, with edge weight being a measure of category similarity between ASINs
copurchaseGraph = networkx.Graph()
for asin, metadata in amazonProducts.items():
    copurchaseGraph.add_node(asin ,SalesRank=metadata["SalesRank"],TotalReviews=metadata['TotalReviews'],AvgRating=metadata['AvgRating'])
    """copurchaseGraph[asin]["SalesRank"] = metadata['SalesRank']
    copurchaseGraph[asin]['TotalReviews']=metadata['TotalReviews']
    copurchaseGraph[asin]['AvgRating']=metadata['AvgRating']
    copurchaseGraph[asin]['DegreeCentrality']=metadata['DegreeCentrality']
    copurchaseGraph[asin]['ClusteringCoeff']=metadata['ClusteringCoeff']"""
    for a in metadata ['Copurchased'].split():
        copurchaseGraph.add_node(a.strip())
        copurchaseGraph.add_edge(asin, a.strip())

In [59]:
copurchaseGraph.number_of_edges()

987942

In [60]:
from networkx.algorithms import community

In [65]:
communities =community_louvain.best_partition(copurchaseGraph)

In [66]:
comun = pd.DataFrame.from_dict([communities])

In [67]:
comun = comun.T

In [68]:
comun.columns = ['modularity_class']
comun['Id'] = comun.index 

In [69]:
comun

,modularity_class,Id
0771044445,0,0771044445
0827229534,1,0827229534
0804215715,1,0804215715
156101074X,1,156101074X
0687023955,1,0687023955
...,...,...
B0000508ZN,372,B0000508ZN
1930519206,246,1930519206
B00006JBIX,21,B00006JBIX
B00008DDST,4476,B00008DDST


In [70]:
comun.reset_index(drop=True)

,modularity_class,Id
0,0,0771044445
1,1,0827229534
2,1,0804215715
3,1,156101074X
4,1,0687023955
...,...,...
548547,372,B0000508ZN
548548,246,1930519206
548549,21,B00006JBIX
548550,4476,B00008DDST


In [71]:
comun.modularity_class.nunique()

187784